### Connect To Workspace

In [9]:
from azureml.core import Workspace

workspace = Workspace.from_config()

Performing interactive authentication. Please follow the instructions on the terminal.
To sign in, use a web browser to open the page https://microsoft.com/devicelogin and enter the code APXF6WJQM to authenticate.
Interactive authentication successfully completed.


### Get dataset and compute target

In [17]:
pokemon_dataset = workspace.datasets['Pokemon Dataset']
compute_target = workspace.compute_targets['k80cluster']

### Create experiment

In [13]:
from azureml.core import Experiment

experiment_name = 'pokemon-classifier'
experiment = Experiment(workspace, name=experiment_name)

### Submit Job

In [23]:
%pycat training/train.py

In [24]:
from azureml.train.dnn import TensorFlow

estimator = TensorFlow(source_directory='training',
                      entry_script='train.py',
                      compute_target=compute_target,
                      script_params = {
                          '--data_dir': pokemon_dataset.as_named_input('pokemon').as_mount(),
                          '--image_dim': 96,
                          '--learning_rate': 0.001,
                          '--batch_size': 32,
                          '--steps_per_epoch': 100,
                          '--num_epochs': 100,
                          '--dropout_rate': 0.5,
                          '--activation_function': 'softmax',
                          '--output_dir':'./outputs',
                      },
                      framework_version='2.0',
                      use_gpu=True,
                      pip_packages=['Pillow==6.2.0', 'scipy==1.1.0', 'azureml-dataprep[fuse,pandas]==1.1.29'])

In [25]:
run = experiment.submit(estimator)

In [26]:
from azureml.widgets import RunDetails
RunDetails(run).show()

_UserRunWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', '…

### Register model

In [ ]:
model = run.register_model(model_name='pokemon-classifier', model_path='outputs')